In [ ]:
!wget -q https://git.io/J0fjL -O IAM_Words.zip
!unzip -qq IAM_Words.zip
!
!mkdir data
!mkdir data/words
!tar -xf IAM_Words/words.tgz -C data/words
!mv IAM_Words/words.txt data

In [7]:
import numpy as np
import os
import pandas as pd
np.random.seed(42)

In [23]:
base_path = "/home/studio-lab-user/data"
words_list = []

words = open(f"{base_path}/words.txt", "r").readlines()
for line in words:
    if line[0] == "#":
        continue
    if line.split(" ")[1] != "err":  # We don't need to deal with errored entries.
        words_list.append(line)

len(words_list)

np.random.shuffle(words_list)

In [24]:
base_image_path = os.path.join(base_path, "words")


def get_image_paths_and_labels(samples):
    paths = []
    corrected_samples = []
    for (i, file_line) in enumerate(samples):
        line_split = file_line.strip()
        line_split = line_split.split(" ")

        # Each line split will have this format for the corresponding image:
        # part1/part1-part2/part1-part2-part3.png
        image_name = line_split[0]
        partI = image_name.split("-")[0]
        partII = image_name.split("-")[1]
        img_path = os.path.join(
            base_image_path, partI, partI + "-" + partII, image_name + ".png"
        )
        if os.path.getsize(img_path):
            paths.append(img_path)
            corrected_samples.append(file_line.split("\n")[0])

    return paths, corrected_samples


img_paths, labels = get_image_paths_and_labels(words_list)

In [25]:
# Find maximum length and the size of the vocabulary in the training data.
labels_cleaned = []
characters = set()
max_len = 0

for label in labels:
    label = label.split(" ")[-1].strip()
    for char in label:
        characters.add(char)

    max_len = max(max_len, len(label))
    labels_cleaned.append(label)

characters = sorted(list(characters))

print("Maximum length: ", max_len)
print("Vocab size: ", len(characters))

Maximum length:  21
Vocab size:  78


In [26]:
df = pd.DataFrame( 
    {'file_path': img_paths,
     'label': labels_cleaned
    })

In [27]:
df.head()

,file_path,label
0,/home/studio-lab-user/data/words/r06/r06-076/r...,or
1,/home/studio-lab-user/data/words/n01/n01-004/n...,unable
2,/home/studio-lab-user/data/words/g06/g06-011f/...,of
3,/home/studio-lab-user/data/words/f04/f04-011/f...,was
4,/home/studio-lab-user/data/words/e04/e04-103/e...,plank


In [29]:
df.to_csv('/home/studio-lab-user/handwriting/handwriting.csv')